# iALM Stage 3: The "Reality Check"

This notebook implements the **Reality Check** described in Section 4.3. 
It connects the asset returns and liability streams to project the evolution of Net Financial Wealth.

## Case Study: The Pimlott Family
* **Initial Wealth:** £382,000 (Liquid)
* **Income:** £115,000/yr (Indexed)
* **Goal:** Sustain lifestyle and pay for education without running out of money (Insolvency).

In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from ialm.engines import iALM_Simulator
from ialm.liabilities import LiabilitySimulator
from ialm.analysis import WealthProjector

# Setup
np.random.seed(101)
STEPS = 240 # 20 Years
SCENARIOS = 500

## 1. Initialize Engines
We simulate a "Balanced Portfolio" (60/40 Equity/Bond proxy) returning ~6% with 10% Vol.

In [ ]:
sim = iALM_Simulator()

# Drivers
inflation_paths = sim.simulate_gou(1.03, 0.03, 0.5, 0.01, STEPS, SCENARIOS)
# Portfolio Returns (Simulated as single asset for simplicity in Reality Check)
portfolio_paths = sim.simulate_gbm(1.0, 0.06, 0.10, STEPS, SCENARIOS)

# Convert Price Path to Returns
portfolio_returns = np.zeros_like(portfolio_paths)
portfolio_returns[0] = 0.0
portfolio_returns[1:] = (portfolio_paths[1:] / portfolio_paths[:-1]) - 1.0

## 2. Build the Household Profile
Using the Pimlott data structure.

In [ ]:
liab_sim = LiabilitySimulator(inflation_paths)
liab_sim.add_liability("Mortgage", 2000, 0, 60, is_indexed=False)     # 5 Years left
liab_sim.add_liability("School", 1500, 36, 84, is_indexed=True, r_add=0.02/12) # Future School
liab_sim.add_liability("Living", 3500, 0, STEPS, is_indexed=True)     # Core Spending

# Income (Salary) - Modeled as a positive inflow stream
inc_sim = LiabilitySimulator(inflation_paths)
inc_sim.add_liability("Salary", 9500, 0, 180, is_indexed=True) # 15 Years until retirement

## 3. Run Wealth Projection
We calculate the "Wealth Cloud" - the distribution of future wealth.

In [ ]:
projector = WealthProjector(
    asset_simulator=sim,
    liability_simulator=liab_sim,
    portfolio_simulator=portfolio_returns
)

wealth_paths = projector.run_simulation(
    initial_wealth=382000,
    income_streams=inc_sim,
    n_steps=STEPS,
    n_scenarios=SCENARIOS
)

## 4. Solvency Analysis
We check the probability that wealth remains positive throughout the period.

In [ ]:
solvency_prob = projector.calculate_solvency(wealth_paths)

plt.figure(figsize=(12, 7))
# Plot Cloud
plt.plot(wealth_paths[:, :50], color='blue', alpha=0.1)
# Plot Mean
plt.plot(np.mean(wealth_paths, axis=1), color='black', linewidth=2, label='Expected Wealth')
# Plot Bankruptcy Line
plt.axhline(0, color='red', linestyle='--', linewidth=2, label='Bankruptcy Threshold')

plt.title(f"Pimlott Household Wealth Evolution\nProbability of Solvency: {solvency_prob*100:.1f}%")
plt.ylabel("Net Financial Wealth (£)")
plt.xlabel("Months")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()